<a href="https://colab.research.google.com/github/KyoldFusion/IMDB-Netflix-/blob/main/NetflixIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
 #import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
ratings = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/ratings.csv"
spark.sparkContext.addFile(ratings)
rf = spark.read.csv(SparkFiles.get("ratings.csv"), sep=",", header=True)

In [ ]:
rf.createOrReplaceTempView('ratings')

In [ ]:
basics = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/titles_basic.csv"
spark.sparkContext.addFile(basics)
bf = spark.read.csv(SparkFiles.get("titles_basic.csv"), sep=",", header=True)

In [ ]:
bf.createOrReplaceTempView('basics')

In [ ]:
names = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(names)
nf = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

In [ ]:
nf.createOrReplaceTempView('names')

In [ ]:
principals = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(principals)
pf = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [ ]:
pf.createOrReplaceTempView('principals')

In [ ]:
spark.sql("Select * from principals").show(5)

+----------+--------+---------+--------+----+-----------------+
|    tconst|ordering|   nconst|category| job|       characters|
+----------+--------+---------+--------+----+-----------------+
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|   actor|null|      "[""Lee""]"|
|tt12189034|       8|nm7672770|   actor|null|"[""Detective""]"|
+----------+--------+---------+--------+----+-----------------+
only showing top 5 rows



In [101]:
spark.sql("Select DISTINCT nconst, primaryname, birthyear, deathyear, CASE WHEN birthyear IS NOT NULL and deathyear IS NOT NULL then CAST(deathyear - birthyear AS decimal(3,0)) ELSE COALESCE((birthyear - deathyear), 2021-birthyear) END  Age, CASE WHEN deathyear IS NULL AND birthyear IS NOT NULL THEN 'alive' WHEN deathyear IS NOT NULL THEN 'deceased' ELSE 'unknown'  END LivingStatus from names where (primaryprofession LIKE '%actor%' OR primaryprofession LIKE '%actress%') AND birthyear IS NOT NULL").show(20)

+---------+--------------------+---------+---------+----+------------+
|   nconst|         primaryname|birthyear|deathyear| Age|LivingStatus|
+---------+--------------------+---------+---------+----+------------+
|nm1441958|           Geddy Lee|     1953|     null|68.0|       alive|
|nm1443104|   Wanda Wilkomirska|     1929|     2018|89.0|    deceased|
|nm1450441|         Lexi Lamour|     1976|     null|45.0|       alive|
|nm1451129| Carles Romero Vidal|     1977|     null|44.0|       alive|
|nm1454258|       Cilly Dartell|     1957|     null|64.0|       alive|
|nm6268945|   Monika Potokárová|     1992|     2019|27.0|    deceased|
|nm6276162|    Alasdair Gillies|     1938|     null|83.0|       alive|
|nm6278631|               Spice|     1982|     null|39.0|       alive|
|nm0000139|        Cameron Diaz|     1972|     null|49.0|       alive|
|nm0000457|           John Hurt|     1940|     2017|77.0|    deceased|
|nm0002168|Willeke van Ammel...|     1944|     null|77.0|       alive|
|nm145

In [99]:
spark.sql("SELECT DISTINCT nconst, primaryname, COALESCE((birthyear - deathyear), 2021-birthyear) age from names WHERE birthyear IS NOT NULL AND nconst = 'nm0459889'").show(10)

AnalysisException: ignored

In [ ]:
spark.sql("Select * from ratings").show(5)

+---------+-------------+--------+
|   tconst|averagerating|numvotes|
+---------+-------------+--------+
|tt0214461|          7.4|      74|
|tt0214659|          6.0|      21|
|tt0214878|          6.1|    1566|
|tt0215244|          3.0|       5|
|tt0215402|          8.1|       9|
+---------+-------------+--------+
only showing top 5 rows



In [ ]:
spark.sql("Select * from basics").show(5)

+---+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|_c0|   tconst|titletype|        primarytitle|       originaltitle|isadult|startyear|endyear|runtimeminutes|           genres|
+---+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|  0|tt0439997|    movie|           500 Almas|           500 Almas|      0|     2004|   null|         105.0|      Documentary|
|  1|tt0439999|tvSpecial|           80s Mania|           80s Mania|      0|     2001|   null|          50.0|            Music|
|  3|tt0440003|    movie|        A-1 Headline|         A1 tou tiao|      0|     2004|   null|          95.0| Mystery,Thriller|
|  4|tt0440004|  tvMovie| AD/BC: A Rock Opera| AD/BC: A Rock Opera|      0|     2004|   null|          30.0|   Comedy,Musical|
|  5|tt0440008|  tvMovie|Abbamania: We Say...|Abbamania: We Say...|      0|     2004|   null|          50.0|Doc

In [ ]:
spark.sql("Select r.tconst from ratings r join basics b on r.tconst = b.tconst join principals p on p.tconst = b.tconst join names n on p.nconst = n.nconst").show(5)

+---------+
|   tconst|
+---------+
|tt1345836|
|tt1345836|
|tt1345836|
|tt1345836|
|tt1345836|
+---------+
only showing top 5 rows

